In [1]:
from mnemonic import Mnemonic
from fastecdsa import keys, curve
import binascii
import hashlib
import hmac


In [2]:
"""here i create a class to store data in a tree like structure"""

class TreeNode():
    
    def __init__(self,parent):
        self.parent=parent
        self.pvt_key=None
        self.pub_key=None
        self.chain_code =None
        self.children = []
    

In [3]:
"""Here I define the function needed to generate the seed which is going to be used later """

def generate_seed(language,passphrase,strenght):
   mnemo = Mnemonic(language)
   words = mnemo.generate(strength=strenght)
   seed = mnemo.to_seed(words, passphrase=passphrase)
   print("Entropy=" + str( binascii.hexlify(mnemo.to_entropy(words)))) #prints the entropy in hex
   print("Words= " + words)   
   print("Seed = "+str(binascii.hexlify(seed))) #prints the seed in hex 
   return seed
   


In [4]:
seed=generate_seed("english","test",128)

Entropy=b'f6561bd1fa2ebfbc4e4884493e9bce75'
Words= wagon radio virtual violin typical taste decrease awesome empty visit vicious tunnel
Seed = b'5b480ff5e76644e23a95ab5448d56869e7d1264308dc99968b3138f8e55c7ccd6e68c2bf12a8fcc65b07af329930cff2c595a810ff349f41edb64621b0340cf5'


In [5]:
def print_info(dictio):
    for keys,values in dictio.items():
        print(keys.upper())
        print(values)
   

In [5]:

def get_master_info(seed):
    master=TreeNode(None)
    master_hmac= hmac.new(seed,None,hashlib.sha512).hexdigest()
    master_pvt_key=master_hmac[:64]
    master.pvt_key=master_pvt_key
    master_chain_code=master_hmac[64:]
    master.chain_code=master_chain_code
    pvt=int(master_pvt_key, 16)
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1)
    pub_key= hex(master_pub_key.y)[2:4]+hex(master_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
    master.pub_key=pub_key
    return master


In [6]:
master_info=get_master_info(seed)
master_info.pub_key


'3c35984165a8c5c8283b5b1b4981347f646ba99abefc0b3adabc27a3bf828ac6af'

In [7]:
"""In this section I define a function to derive child addresses using hardened derivation"""

def create_hardened_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pvt_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)
            

In [8]:
"""in this section I create a children from the public key of the master"""

def create_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pub_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)

In [9]:
create_children(master_info,"00000000",2)
create_hardened_children(master_info,"80000000",1)

for i in master_info.children:
        print("Private Key " + i.pvt_key)
        print("Public Key " + i.pub_key)
        print("Chain Code " + i.chain_code)
        print("-------------------------------------------------------------------------")
        

Private Key 9caef8c0ddbfcdfde861dc1c637d0b714c2b6ef2c5cf28658a3004a2ce656ddf
Public Key c0a5c2d2f6b48484b0b0e4d06f803c8c2d5c5777a9251da7800b742455b228b048
Chain Code e29e277d53cbbbaf5f1706ffb089b8c148b8453a33ff8d45f299afc74e7766d8
-------------------------------------------------------------------------
Private Key ce217f1f07c3aea4080119828989aa48bfcbbb674ebdec93687ee36c6cf7c381
Public Key 17dabcee49cae895c882e873dbebc46bec694e04e24703cb792848581aaacdd44f
Chain Code 9df8a0cc23e4fa2279ee4bc70e53db8f31c38813971d443aefbec6748a15f0bd
-------------------------------------------------------------------------
Private Key 9ddbce744ff07a3741de03e5c3cf65d2d3b0d3425d8e88b7d488e59a1f1c645c
Public Key 49e262a4f2415e3656ab5a6df1cf78c56142d7dbc4e6bdf08eb77865f9f8a94e14
Chain Code 9e20817960cacac6f393d608ad844fbdf80c8d8d0762b2781b4bbeb5fc7c41a3
-------------------------------------------------------------------------


In [10]:
"""Lets try to create some grandkids"""

for i in master_info.children:
    create_hardened_children(i,"80000000",1)
    create_children(i,"00000000",2)
    for j in i.children:
        print("Private Key " + j.pvt_key)
        print("Public Key " + j.pub_key)
        print("Chain Code " + j.chain_code)
        print("-------------------------------------------------------------------------")
        




Private Key 461c58b739fb663fb216bf5529a3cf85f36861d1de9876a57df1db916698db8b
Public Key ab366ee88edd3dbcf33bfc55644b21bfec924ac7521e24d9933cf4eae345581eda
Chain Code 3bc2a7f959e3feed51deffdd3258e85bc59ea33b2a17d1ee5c0416ce608dec26
-------------------------------------------------------------------------
Private Key 8d67d05cdd40ba6c0c841519a06cb0c324078bd228881a4669f842fb7246a068
Public Key bba6bc18d30281df30d3b4203572165627a84a3a4ff00d7e47cb89aa5c581982fb
Chain Code fd5d6d225c688dab2e40ed3b67f74ee77ed4d2fbf17520ec427957ea4560cc64
-------------------------------------------------------------------------
Private Key a0f12705ec1cf3bf9a2f248e381a2c2664e2c5a65292721b1259195af641ee29
Public Key 1a66b21e4b0aa510d83003553570cacd64d34363e5c959368bed32015b7d09a0b7
Chain Code 72210090b84f9e5a01d719a15dc950e6ff7986a9311a176434dc2fdba621173f
-------------------------------------------------------------------------
Private Key fbed8c7a1d82a83a3a86de68178fa9a9f50c54c661779e342795caec06775138
Public K